In [ ]:
!pip install transformers accelerate bitsandbytes --quiet


In [ ]:
from huggingface_hub import login
login("YOUR-SECRET-TOKEN")

In [ ]:
model_id = "tiiuae/falcon-7b-instruct"

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")





In [ ]:
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7
)


In [ ]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

In [ ]:
from google.colab import files
uploaded = files.upload()  # carica cluster_descriptions.json e theme_daily_sentiment.json


In [ ]:
uploaded = files.upload()  # carica  theme_daily_sentiment.json


In [ ]:
uploaded = files.upload()  # carica  cluster_entity.json

In [ ]:
import json

with open("cluster_descriptions.json", "r") as f:
    cluster_labels = json.load(f)

with open("theme_daily_sentiment.json", "r") as f:
    daily_sentiment = json.load(f)

with open("cluster_entity_likes.json", "r") as f:
    entity_likes = json.load(f)


In [ ]:
import json
from langchain.prompts import PromptTemplate
from collections import defaultdict

# === Carica i file JSON ===
with open("cluster_descriptions.json", "r", encoding="utf-8") as f:
    cluster_labels = json.load(f)

with open("theme_daily_sentiment.json", "r", encoding="utf-8") as f:
    daily_sentiment = json.load(f)

with open("cluster_entity_likes.json", "r", encoding="utf-8") as f:
    entity_likes_by_cluster = json.load(f)

# === Organizza i dati giornalieri per cluster ===
sentiment_by_cluster = defaultdict(list)
for entry in daily_sentiment:
    sentiment_by_cluster[str(entry["theme_cluster"])].append(entry)

# === Prompt template compatto ===
cluster_prompt = PromptTemplate.from_template(
    """You are a social media analyst for the Monaco GP 2025.

Topic: {topic_label}

Recent sentiment trend:
{daily_scores}

Most mentioned entities (by total likes):
{entity_likes}

Write 2-3 sentences summarizing how the sentiment evolved over time.
Then, comment on which drivers or teams got the most attention.
Do not invent events or motivations.
Use only the data provided above."""
)


# === Parametri di sicurezza ===

cluster_id = "2"
max_days = 5
max_entities = 3

# === Dati del cluster ===
label = cluster_labels[cluster_id]
trend = sorted(sentiment_by_cluster[cluster_id], key=lambda x: x["date"])[-max_days:]
daily_scores = "\n".join(
    f"- {d['date']}: sentiment {d['avg_roberta_pos']:.2f}" for d in trend
)

# === Entità più menzionate (con like) ===
entity_likes = entity_likes_by_cluster.get(cluster_id, {})
if entity_likes:
    sorted_entities = sorted(entity_likes.items(), key=lambda x: x[1], reverse=True)[:max_entities]
    entity_likes_str = "\n".join(f"- {name}: {likes} likes" for name, likes in sorted_entities)
else:
    entity_likes_str = "No major entities mentioned."

# === Costruisci il prompt finale
prompt = cluster_prompt.format(
    topic_label=label,
    daily_scores=daily_scores,
    entity_likes=entity_likes_str
)

# === Genera output con pipeline Hugging Face
result = generator(prompt)[0]["generated_text"]

# === Stampa
print(f"\n {label}\n{result.strip()}")
